In [ ]:
!pip install matplotlib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
test = pd.read_csv("../input/store-sales-time-series-forecasting/test.csv")
train = pd.read_csv("../input/store-sales-time-series-forecasting/train.csv")
stores = pd.read_csv("../input/store-sales-time-series-forecasting/stores.csv")
transactions = pd.read_csv("../input/store-sales-time-series-forecasting/transactions.csv")
oil = pd.read_csv("../input/store-sales-time-series-forecasting/oil.csv")
holiday_events = pd.read_csv("../input/store-sales-time-series-forecasting/holidays_events.csv")

In [ ]:
train['date'] = pd.to_datetime(train.date)

In [ ]:
train.info()

In [ ]:
transactions['date'] = pd.to_datetime(transactions.date)

In [ ]:
transactions.info()

In [ ]:
oil['date'] = pd.to_datetime(oil.date)

In [ ]:
oil.info()

In [ ]:
train.tail()
#Onpromotion = how many units sold on promottion at that date.

In [ ]:
train.groupby(['store_nbr']).sales.sum()

In [ ]:
train.store_nbr.unique()

In [ ]:
stores.tail()
#Cluster is grouping similar stores.

In [ ]:
transactions.tail()

In [ ]:
oil.tail()

In [ ]:
stores.tail()

In [ ]:
oil.isnull().sum()

It looks like we are missing 43 oil prices.

In [ ]:
stores.isnull().sum()

In [ ]:
transactions.isnull().sum()

In [ ]:
train.isnull().sum()

### Handling the missing values for Oil Prices

In [ ]:
oil.plot(x= 'date', y= "dcoilwtico")

In [ ]:
oil_new=(oil["dcoilwtico"].interpolate(method='linear', axis=0))

In [ ]:
oil_new.plot(x= 'date',y='dcoilwtico')

In [ ]:
oil

In [ ]:
oil_new

### 1.2  How the data is distributed?

- Plotting outliers
- Checking for distributions. (Which groups are represented heavily on the datasets?)
- Check for impossible values. (like oil prices: male or oil price: -123132)

We have 5 datasets consisting of train, holiday_events, oil_new, stores and transactions. Now that we need to check which distribution do they follow,what are the correlations between features and we will have more idea about the data we have on hand.

##### 1.2.1 Train:

Lets check the number of zeros in the dataset.

In [ ]:
(train==0).sum()

In [ ]:
train.plot(x='date',y='sales')

To give us a point between 2013 and 2017. almost every year there is a spike in the sales at the end of the year.
In 2016. the sales hits its highest levels around the first quarter and at the last quarter. These two spikes needs to be investigated as well as the usual spikes ath the end of each year.

In [ ]:
train.plot(x= 'store_nbr', y= 'sales')

Store Number 1, Number 9, Number 20, Number 39 and Number 45 are provided the highest total sales throughout the 4-year period.

In [ ]:
train.plot(x='store_nbr', y='onpromotion')

Checking the highest promotional sales numbers gives us some insight about how much promotion impact on several number of stores sales'.
Here store Number 16, Number 33, Number 35, Number 36, Number 53 and Number 54 are the ones that are the most benefitted  by the promotion campaigns. This means promotions might be playing a fair amount of role in the sales of these stores. Or this can be like, they already have a huge sale because they are maybe in the city centre (more reachable) or more customer waiting for them, maybe they are based in the metropolitan cities or they made the most discounts on the price. We will consider these factors later on.

In [ ]:
train.plot(x='date',y='onpromotion')

This plot explains the reason why we have a huge number of zero values in 'onpromotion' feature. Until at the end of first quarter of 2014, company did not gain any sales from promotion or it did not run a campaign. There are also some spikes aroung 2016 and 2017 in terms of promotional sales.

##### We need to look into how the promotion affects the total sales. How much percentage of sales is actually coming from promotional sales?

In [ ]:
sale_percentage = train['onpromotion']/train['sales']

In [ ]:
a=pd.DataFrame(sale_percentage,columns= ["salepercentage"])

In [ ]:
s_pro = pd.concat([train,a],axis=1)

In [ ]:
s_pro

In [ ]:
s_pro.plot(x= "date",y="salepercentage")

Lastly, lets look at how the product families impact on sales.

In [ ]:
train.plot(x="family",y="sales")

In [ ]:
train['family'].unique()

In [ ]:
train.plot(x='family',y='onpromotion')

- As you can see, some of the product families had not been promoted.
- Why there is no promotional sales on Automotive, Appliances,etc?
- In some products ,promotion is more effective since they have higher sales.

#### 1.2.2  Oil

oil_new was the interpolated oil dataset. We are going to use this interpolated data now.
First let's check the number of zeros in the dataset. Oil price should not have a zero value unless there is an explanation.

In [ ]:
(oil_new==0).sum().sum()

In [ ]:
oil_new= pd.DataFrame(oil_new)

In [ ]:
oil_new['date']= oil['date']

In [ ]:
oil_new

In [ ]:
oil_new['dcoilwtico'].interpolate(method='linear',axis=0)

Here, I do not know why index zero equals to 1. I am going to delete that value for now, since I did not find a way to interpolate that.

In [ ]:
del oil_new['dcoilwtico'][0]

In [ ]:
oil_new.plot(x= 'date', y= 'dcoilwtico')

- Roughly, throughout this 5 year period, we see a downwards trend after the last quarter of 2014. What happened at that time? How the sales were affected from this downwards trend?
- How the sales affected when the oil prices goes hits rock bottom at the begining of 2016.

Oil Prices and Sales

In [ ]:
za = train.groupby(['date']).sum()

In [ ]:
za = pd.DataFrame(za)

In [ ]:
oil_new['date'] = pd.to_datetime(oil_new.date)
oil_new.set_index('date')

In [ ]:
ba = pd.concat([oil_new, za],axis =1)

In [ ]:
za = pd.DataFrame(za)

In [ ]:
za

In [ ]:
ba = pd.concat([oil_new, za],axis =1)

In [ ]:
tata = za.merge(oil_new, on='date')

In [ ]:
tata

In [ ]:
tata.plot(kind = 'scatter' ,x= 'dcoilwtico', y= 'sales')

In [ ]:
tata.corr()

In [ ]:
# create figure and axis objects with subplots()
fig,ax = plt.subplots()
# make a plot
ax.plot(tata.date, tata.sales, color="Red", marker="o")
# set x-axis label
ax.set_xlabel("Year",fontsize=14)
# set y-axis label
ax.set_ylabel("Sales",color="red",fontsize=14)


ax2=ax.twinx()
# make a plot with different y-axis using second axis object
ax2.plot(tata.date, tata.dcoilwtico,color="blue",marker="o")
ax2.set_ylabel("Oil Price",color="blue",fontsize=14)
plt.show()

I do not see a strong correlation between sales and oil price for now. Also the corr() output indicates that we do not have a negative correlation between sales and oil price.
Indeed, when the oil price was plummeted, sales were rising but it does not have a sharp rise as you would expect.

In [ ]:
# create figure and axis objects with subplots()
fig,ax = plt.subplots()
# make a plot
ax.plot(tata.date, tata.onpromotion, color="Red", marker="o")
# set x-axis label
ax.set_xlabel("Year",fontsize=14)
# set y-axis label
ax.set_ylabel("Onpromotion",color="red",fontsize=14)


ax2=ax.twinx()
# make a plot with different y-axis using second axis object
ax2.plot(tata.date, tata.dcoilwtico,color="blue",marker="o")
ax2.set_ylabel("Oil Price",color="blue",fontsize=14)
plt.show()

The impact of declining oil prices to the promotional sales.